In [75]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from collections import Counter

from heapq import nlargest
import numpy as np
import scipy as sp
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

from tqdm import tqdm
# tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

def get_mode(row): # Кастомная функция вычисления модального значения категориального признака
    counter = Counter(row)
    (most_common, _) = counter.most_common(1)[0]

    return most_common

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [ ]:
from datetime import datetime
import pytz

region_timezones = {
    'Moscow': 'Europe/Moscow',                       # +03:00
    'Moscow Oblast': 'Europe/Moscow',                # +03:00
    'St.-Petersburg': 'Europe/Moscow',               # +03:00
    'Sverdlovsk Oblast': 'Asia/Yekaterinburg',       # +05:00
    'Sverdlovsk': 'Asia/Yekaterinburg',              # +05:00
    'Krasnodar Krai': 'Europe/Moscow',               # +03:00
    'Novosibirsk Oblast': 'Asia/Novosibirsk',        # +07:00
    'Bashkortostan Republic': 'Asia/Yekaterinburg',  # +05:00
    'Samara Oblast': 'Europe/Samara',                # +04:00
    'Nizhny Novgorod Oblast': 'Europe/Moscow',       # +03:00
    'Chelyabinsk': 'Asia/Yekaterinburg',             # +05:00
    'Tatarstan Republic': 'Europe/Moscow',           # +03:00
    'Rostov': 'Europe/Moscow',                       # +03:00
    'Krasnoyarsk Krai': 'Asia/Krasnoyarsk',          # +07:00
    'Perm Krai': 'Asia/Yekaterinburg',               # +05:00
    'Leningradskaya Oblast\'': 'Europe/Moscow',      # +03:00
    'Yaroslavl Oblast': 'Europe/Moscow',             # +03:00
    'Irkutsk Oblast': 'Asia/Irkutsk',                # +08:00
    'Saratov Oblast': 'Europe/Saratov',              # +04:00
    'Voronezh Oblast': 'Europe/Moscow',              # +03:00
    'Stavropol Kray': 'Europe/Moscow',               # +03:00
    'Primorye': 'Asia/Vladivostok',                  # +10:00
    'Khabarovsk': 'Asia/Vladivostok',                # +10:00
    'Volgograd Oblast': 'Europe/Volgograd',          # +03:00
    'Amur Oblast': 'Asia/Yakutsk',                   # +09:00
    'Khanty-Mansia': 'Asia/Yekaterinburg',           # +05:00
    'Kuzbass': 'Asia/Novokuznetsk',                  # +07:00
    'Omsk Oblast': 'Asia/Omsk',                      # +06:00
    'Tula Oblast': 'Europe/Moscow',                  # +03:00
    'Udmurtiya Republic': 'Europe/Samara',           # +04:00
    'Orenburg Oblast': 'Asia/Yekaterinburg',         # +05:00
    'Altay Kray': 'Asia/Barnaul',                    # +07:00
    'Tomsk Oblast': 'Asia/Tomsk',                    # +07:00
    'Khakasiya Republic': 'Asia/Krasnoyarsk',        # +07:00
    'Vladimir Oblast': 'Europe/Moscow',              # +03:00
    'Tver Oblast': 'Europe/Moscow',                  # +03:00
    'Tyumen Oblast': 'Asia/Yekaterinburg',           # +05:00
    'Belgorod Oblast': 'Europe/Moscow',              # +03:00
    'Ryazan Oblast': 'Europe/Moscow',                # +03:00
    'Kaliningrad Oblast': 'Europe/Kaliningrad',      # +02:00
    'Ulyanovsk': 'Europe/Samara',                    # +04:00
    'Arkhangelskaya': 'Europe/Moscow',               # +03:00
    'Chuvashia': 'Europe/Moscow',                    # +03:00
    'Lipetsk Oblast': 'Europe/Moscow',               # +03:00
    'Vologda Oblast': 'Europe/Moscow',               # +03:00
    'Novgorod Oblast': 'Europe/Moscow',              # +03:00
    'Murmansk': 'Europe/Moscow',                     # +03:00
    'Smolensk Oblast': 'Europe/Moscow',              # +03:00
    'Buryatiya Republic': 'Asia/Irkutsk',            # +08:00
    'Tambov Oblast': 'Europe/Moscow',                # +03:00
    'Bryansk Oblast': 'Europe/Moscow',               # +03:00
    'Komi': 'Europe/Moscow',                         # +03:00
    'Kirov Oblast': 'Europe/Moscow',                 # +03:00
    'Kemerovo Oblast': 'Asia/Novokuznetsk',          # +07:00
    'Adygeya Republic': 'Europe/Moscow',             # +03:00
    'Sakha': 'Asia/Yakutsk',                         # +09:00
    'Kostroma Oblast': 'Europe/Moscow',              # +03:00
    'Astrakhan Oblast': 'Europe/Volgograd',          # +03:00
    'Kamchatka': 'Asia/Kamchatka',                   # +12:00
    'Sakhalin Oblast': 'Asia/Sakhalin',              # +11:00
    'Zabaykalskiy (Transbaikal) Kray': 'Asia/Chita', # +09:00
    'Yamalo-Nenets': 'Asia/Yekaterinburg',           # +05:00
    'Dagestan': 'Europe/Moscow',                     # +03:00
    'Karelia': 'Europe/Moscow',                      # +03:00
    'Pskov Oblast': 'Europe/Moscow',                 # +03:00
    'Kursk Oblast': 'Europe/Moscow',                 # +03:00
    'Kursk': 'Europe/Moscow',                        # +03:00
    'Oryol oblast': 'Europe/Moscow',                 # +03:00
    'Mordoviya Republic': 'Europe/Moscow',           # +03:00
    'Mariy-El Republic': 'Europe/Moscow',            # +03:00
    'North Ossetia–Alania': 'Europe/Moscow',         # +03:00
    'Vladimir': 'Europe/Moscow',                     # +03:00
    'Saratovskaya Oblast': 'Europe/Saratov',         # +04:00
    'Tula': 'Europe/Moscow',                         # +03:00
    'Voronezj': 'Europe/Moscow',                     # +03:00
    'Chukotka': 'Asia/Anadyr',                       # +12:00
    'Crimea': 'Europe/Simferopol',                   # +03:00
    'Kalmykiya Republic': 'Europe/Volgograd',        # +03:00
    'Tyva Republic': 'Asia/Krasnoyarsk',             # +07:00
    'Jewish Autonomous Oblast': 'Asia/Vladivostok',  # +10:00
    'Transbaikal Territory': 'Asia/Chita',           # +09:00
    'Ingushetiya Republic': 'Europe/Moscow',         # +03:00
    'Omsk': 'Asia/Omsk',                             # +06:00
    'Stavropol’ Kray': 'Europe/Moscow',              # +03:00
    'Arkhangelsk Oblast': 'Europe/Moscow',           # +03:00
    'Astrakhan': 'Europe/Volgograd',                 # +03:00
    'Penza Oblast': 'Europe/Moscow',
    'Kurgan Oblast': 'Asia/Yekaterinburg',
    'Kaluga Oblast': 'Europe/Moscow',
    'Ivanovo Oblast': 'Europe/Moscow',
    'Krasnodarskiy': 'Europe/Moscow',
    'Krasnoyarskiy Krai': 'Asia/Krasnoyarsk',  # Добавлен Красноярский край
    'Krasnoyarskiy': 'Asia/Krasnoyarsk',  # Добавлен Красноярский край
    'Chechnya': 'Europe/Moscow',
    'Karachayevo-Cherkesiya Republic': 'Europe/Moscow',
    'Magadan Oblast': 'Asia/Magadan',
    'Altai': 'Asia/Barnaul',  # Для обозначения обоих регионов
    'Ivanovo': 'Europe/Moscow',  # Иваново
    'Jaroslavl': 'Europe/Moscow',  # Ярославль (обычно "Yaroslavl" на английском)
    'Kabardino-Balkariya Republic': 'Europe/Moscow',  # Кабардино-Балкария
    'Kaliningrad': 'Europe/Kaliningrad',  # Калининград
    'Kaluga': 'Europe/Moscow',  # Калуга
    'Kirov': 'Europe/Moscow',  # Киров
    'Nenets': 'Europe/Moscow',  # Ненецкий автономный округ
    'North Ossetia': 'Europe/Moscow',  # Северная Осетия
    'Orel Oblast': 'Europe/Moscow',  # Орловская область
    'Penza': 'Europe/Moscow',  # Пенза
    'Perm': 'Asia/Yekaterinburg',  # Пермь (Пермский край)
    'Primorskiy (Maritime) Kray': 'Asia/Vladivostok',  # Приморский край
    'Sebastopol City': 'Europe/Simferopol',  # Севастополь
    'Smolensk': 'Europe/Moscow',  # Смоленск
    'Smolenskaya Oblast’': 'Europe/Moscow',  # Смоленская область
    'Stavropol Krai': 'Europe/Moscow',  # Ставропольский край
    'Tambov': 'Europe/Moscow',  # Тамбов
    'Tver’ Oblast': 'Europe/Moscow',  # Тверская область
    'Tyumen’ Oblast': 'Asia/Yekaterinburg',  # Тюменская область
    'Vologda': 'Europe/Moscow',  # Вологда
}

# Функция для перевода времени
def convert_to_local_time(row):
    # Парсим timestamp
    event_time = pd.to_datetime(row['event_timestamp'])

    # Получаем часовой пояс региона
    region_name = row['region']
    if region_name in region_timezones:
        timezone = pytz.timezone(region_timezones[region_name])
    else:
        raise ValueError(f"Unknown region: {region_name}")

    # Переводим в местное время
    local_time = event_time.astimezone(timezone)
    local_time = local_time.strftime('%Y-%m-%d %H:%M:%S')

    return local_time

In [2]:
path = './' # Ваш путь до директории с данными /path/to/data/
data = pd.read_csv(path + 'train_events.csv')
video = pd.read_csv(path + 'video_info_v2.csv')
targets = pd.read_csv(path + 'train_targets.csv')

# Предсказание социально-демографических характеристик пользователей Rutube
    Пользователи RUTUBE не всегда указывают свои данные, такие как возраст и пол, что затрудняет формирование портрета пользователя и создание персонализированных рекомендаций. Это ограничивает возможности платформы в предоставлении контента, который наиболее подходит интересам и потребностям пользователей, тем самым ухудшая пользовательский опыт.

    Необходимо разработать модель, которая на основе истории просмотров сможет предсказывать пол и возраст пользователя. В качестве baseline решения мы предоставляем этот ноутбук.


In [3]:
TRAIN_IDS, VAL_IDS = train_test_split(data['viewer_uid'].unique(), train_size=0.8, shuffle=True, random_state=11)

# Categorial EDA 

In [110]:
data['timezone'] = pd.Series(region_timezones).loc[data['region']].values
data['local_event_timestamp'] = data.apply(convert_to_local_time, 1)
data['weekday'] = pd.to_datetime(data['local_event_timestamp']).dt.weekday % 5
data['part of day'] = pd.to_datetime(data['local_event_timestamp']).dt.hour % 6

In [101]:
events = data.merge(video, 'left')
train_events = events[events['viewer_uid'].isin(TRAIN_IDS)].merge(targets, on='viewer_uid', how='inner')
val_targets = targets[targets['viewer_uid'].isin(VAL_IDS)]
age_class_bins = [9, 20, 30, 40, 60] # Возрастные категории пользователей, подробнее в файле с описанием данных
val_targets.loc[:, 'age_class'] = pd.cut(val_targets['age'], bins=age_class_bins, labels=[0, 1, 2, 3])
val_targets = val_targets.sort_values(by='viewer_uid').reset_index(drop=True)

/var/folders/1j/52ghjy5x1v13yrj1b6fgwlcc0000gn/T/ipykernel_33550/105563554.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_targets.loc[:, 'age_class'] = pd.cut(val_targets['age'], bins=age_class_bins, labels=[0, 1, 2, 3])


In [184]:
def check(predicts):
    age_class_bins = [0, 20, 30, 40, 60] # Возрастные категории пользователей, подробнее в файле с описанием данных
    predicts['age_class'] = pd.cut(predicts['age'], bins=age_class_bins, labels=[0, 1, 2, 3])
    predicts = predicts.sort_values(by='viewer_uid').reset_index(drop=True)

    f1_weighted = f1_score(val_targets['age_class'], predicts['age_class'], average='weighted')
    accuracy = accuracy_score(val_targets['sex'], predicts['sex'])

    final_score = 0.7 * f1_weighted + 0.3 * accuracy
    print(f'Weighted F1 = {f1_weighted:.4f} \tAccuracy = {accuracy:.4f} \tFinal Score = {final_score:.4f}')

In [126]:
def aggregate_mode(cat):
    info = pd.DataFrame({'age': train_events.groupby(cat)['age'].median(), 'sex': train_events.groupby(cat)['sex'].apply(get_mode)})
    val_events = events[events['viewer_uid'].isin(VAL_IDS)].join(info, on=cat, how='left')
    predicts = pd.DataFrame({'age': val_events.groupby('viewer_uid')['age'].median(), 'sex': val_events.groupby('viewer_uid')['sex'].apply(get_mode)}).loc[VAL_IDS]
    predicts['sex'] = predicts['sex'].fillna(info['sex'].mode()[0])
    predicts['age'] = predicts['age'].fillna(info['age'].median())
    return predicts

In [95]:
check(aggregate_mode('author_id'))

Weighted F1 = 0.3630 	Accuracy = 0.7331 	Final Score = 0.4740


In [96]:
check(aggregate_mode('category'))

Weighted F1 = 0.2205 	Accuracy = 0.6884 	Final Score = 0.3609


In [97]:
check(aggregate_mode('rutube_video_id'))

Weighted F1 = 0.3970 	Accuracy = 0.7408 	Final Score = 0.5002


In [102]:
check(aggregate_mode('timezone'))

Weighted F1 = 0.1914 	Accuracy = 0.4967 	Final Score = 0.2830


In [103]:
check(aggregate_mode('region'))

Weighted F1 = 0.1989 	Accuracy = 0.4971 	Final Score = 0.2884


In [105]:
check(aggregate_mode('ua_device_type'))

Weighted F1 = 0.1914 	Accuracy = 0.4967 	Final Score = 0.2830


In [107]:
check(aggregate_mode('ua_client_type'))

Weighted F1 = 0.1914 	Accuracy = 0.4967 	Final Score = 0.2830


In [108]:
check(aggregate_mode('ua_os'))

Weighted F1 = 0.2523 	Accuracy = 0.4990 	Final Score = 0.3263


In [109]:
check(aggregate_mode('ua_client_name'))

Weighted F1 = 0.2458 	Accuracy = 0.5278 	Final Score = 0.3304


In [111]:
check(aggregate_mode('weekday'))

Weighted F1 = 0.1914 	Accuracy = 0.4967 	Final Score = 0.2830


In [112]:
check(aggregate_mode('part of day'))

Weighted F1 = 0.1914 	Accuracy = 0.4967 	Final Score = 0.2830


categorial by mode

- yes: rutube_video_id, category, author_id, 

- mb: ua_os, ua_client_name, age stds

- no: ua_client_type, ua_device_type, timezone, region, weekday, part of day

In [190]:
def aggregate_mode_train(cat):
    info = pd.DataFrame({'age': train_events.groupby(cat)['age'].median(), 'sex': train_events.groupby(cat)['sex'].apply(get_mode)})
    _events = events[events['viewer_uid'].isin(TRAIN_IDS)].join(info, on=cat, how='left')
    predicts = pd.DataFrame({
            cat+' age': _events.groupby('viewer_uid')['age'].median(),
            # cat+' var age': _events.groupby('viewer_uid')['age'].std(),
            cat+' sex': _events.groupby('viewer_uid')['sex'].apply(get_mode)
        }).loc[TRAIN_IDS]
    predicts[cat+' age'] = predicts[cat+' age'].fillna(info['age'].median())
    # predicts[cat+' var age'] = predicts[cat+' var age'].fillna(0)
    predicts[cat+' sex'] = predicts[cat+' sex'].fillna(info['sex'].mode()[0])
    return predicts
def aggregate_mode_val(cat):
    info = pd.DataFrame({'age': train_events.groupby(cat)['age'].median(), 'sex': train_events.groupby(cat)['sex'].apply(get_mode)})
    _events = events[events['viewer_uid'].isin(VAL_IDS)].join(info, on=cat, how='left')
    predicts = pd.DataFrame({
            cat+' age': _events.groupby('viewer_uid')['age'].median(),
            # cat+' var age': _events.groupby('viewer_uid')['age'].std(),
            cat+' sex': _events.groupby('viewer_uid')['sex'].apply(get_mode)
        }).loc[VAL_IDS]
    predicts[cat+' age'] = predicts[cat+' age'].fillna(info['age'].median())
    # predicts[cat+' var age'] = predicts[cat+' var age'].fillna(0)
    predicts[cat+' sex'] = predicts[cat+' sex'].fillna(info['sex'].mode()[0])
    return predicts
# data_mode = pd.DataFrame({c: data[c].groupby(data['viewer_uid']).apply(get_mode) for c in ['ua_client_type', 'ua_device_type', 'weekday', 'part of day']})

from catboost import CatBoostClassifier, CatBoostRegressor
X_tr = pd.concat((
    aggregate_mode_train('rutube_video_id'),
    aggregate_mode_train('category'),
    aggregate_mode_train('author_id'),
    aggregate_mode_train('ua_os'),
    aggregate_mode_train('ua_client_name'),
    ), 1)
X_vl = pd.concat((
    aggregate_mode_val('rutube_video_id'),
    aggregate_mode_val('category'),
    aggregate_mode_val('author_id'),
    aggregate_mode_val('ua_os'),
    aggregate_mode_val('ua_client_name'),
    ), 1)

y_tr = targets.set_index('viewer_uid')[['sex','age']].loc[TRAIN_IDS]
lsex = LabelEncoder().fit(y_tr['sex'])
y_tr['sex'] = lsex.transform(y_tr['sex'])
cf = [c+' sex' for c in ['rutube_video_id', 'category', 'author_id', 'ua_client_name', 'ua_os']]  #+ ['ua_client_type', 'ua_device_type', 'weekday', 'part of day']
model_sex = CatBoostClassifier(verbose=False, cat_features=cf).fit(X_tr, y_tr['sex'])
model_age = CatBoostRegressor(verbose=False, cat_features=cf).fit(X_tr, y_tr['age'])

In [161]:
check(pd.DataFrame({'sex': lsex.inverse_transform(model_sex.predict(X_vl)), 'age': model_age.predict(X_vl), 'viewer_uid': X_vl.index}))

Weighted F1 = 0.4194 	Accuracy = 0.7425 	Final Score = 0.5163


# Подсчет метрики

In [106]:
data.head(1)

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,local_event_timestamp,weekday,part of day,timezone
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243,2024-06-01 08:40:58,0,0,Asia/Yekaterinburg


# Пример заполнения submission.csv
    Перед отправкой убедитесь, что количество уникальных пользователей в тестовой выборке и в вашей таблице предсказаний совпадает. Проверьте, чтобы в столбцах с прогнозируемыми значениями не было пропущенных значений. 

    Идентификаторы пользователей (viewer_uid) нужно сохранить как отдельную колонку, а не индекс таблицы.
    
    Не забудьте сохранить файл с параметром index = False.

In [21]:
submission = pd.DataFrame(columns=['viewer_uid', 'sex', 'age_class'])
submission['viewer_uid'] = predicts['viewer_uid'].values
submission['sex'] = predicts['sex_mode'].values
submission['age_class'] = predicts['age_class'].values

In [22]:
submission.head()

,viewer_uid,sex,age_class
0,10000001,female,2
1,10000007,male,2
2,10000010,male,2
3,10000012,male,2
4,10000014,male,2


In [ ]:
submission.to_csv(
    # path/submission.csv,
    # index=False
)